In [1]:
import fiftyone as fo
import fiftyone.brain as fob
import clip
import torch

import numpy as np
from PIL import Image
from tqdm import tqdm

# CLIP 모델 로드
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

## 필요한 기능 함수 정의
# 개별 데이터 태깅, 메타데이터 추가
def add_tags(dataset, source):
    
    for sample in dataset:
        sample.tags.append("image")
        sample['source'] = source
        sample.save()

# 임베딩 생성 함수 : 이미지, 텍스트 구분
def get_embeddings(sample):

    if "image" in sample.tags:
        with torch.no_grad():
            inputs = preprocess(Image.open(sample.filepath)).unsqueeze(0).to(device)
            features = model.encode_image(inputs)

    # elif sample.tags[1] == "text":
    #     with torch.no_grad():
    #         inputs = clip.tokenize(sample.original_text, context_length=77, truncate=True).to(device)
    #         features = model.encode_text(inputs)

    return features.cpu().numpy().flatten()

# 개별 데이터에 대한 임베딩 추출 및 임베딩 매핑 리스트 생성
def add_embeddings(dataset):

    print(f"{dataset}")
    for sample in tqdm(dataset, desc=f"{dataset.name} 임베딩 추출 중"):
        embedding = get_embeddings(sample)
        sample['clip_embeddings'] = embedding.tolist()
        sample.save()

def collect_image_embeddings_by_sample_id(dataset):

    image_embeddings = {}
    for sample in dataset:
        sample_id = sample.id
        if 'clip_embeddings' in sample:
            # 샘플 ID를 키로, 이미지 임베딩을 값으로 저장
            image_embeddings[sample_id] = np.array(sample['clip_embeddings'])

    return image_embeddings

fo.list_datasets()

['LPDR_cur2',
 'LPDR_merged',
 'LPDR_ref2',
 'LPDR_test2',
 'kolp_dataset',
 'ls-annotation-example',
 'test_raw_imgs',
 'tunelpdr_cur',
 'tunelpdr_merged',
 'tunelpdr_ref',
 'tunelpdr_test']

In [6]:
cur_dataset = fo.load_dataset("tunelpdr_test")
cur_dataset

Name:        tunelpdr_test
Media type:  image
Num samples: 10000
Persistent:  False
Tags:        []
Sample fields:
    id:               fiftyone.core.fields.ObjectIdField
    filepath:         fiftyone.core.fields.StringField
    tags:             fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:         fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    created_at:       fiftyone.core.fields.DateTimeField
    last_modified_at: fiftyone.core.fields.DateTimeField
    ground_truth:     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    source:           fiftyone.core.fields.StringField

In [16]:
cur_dataset.first()['clip_embeddings']

KeyError: "Sample has no field 'clip_embeddings'"

In [7]:
test_dataset = fo.load_dataset("tunelpdr_cur")
test_dataset

Name:        tunelpdr_cur
Media type:  image
Num samples: 343
Persistent:  False
Tags:        []
Sample fields:
    id:               fiftyone.core.fields.ObjectIdField
    filepath:         fiftyone.core.fields.StringField
    tags:             fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:         fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    created_at:       fiftyone.core.fields.DateTimeField
    last_modified_at: fiftyone.core.fields.DateTimeField
    ground_truth:     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    source:           fiftyone.core.fields.StringField

In [15]:
test_dataset.first()['clip_embeddings']

KeyError: "Sample has no field 'clip_embeddings'"

In [14]:
image_embeddings_by_sample_id = collect_image_embeddings_by_sample_id(test_dataset)

# 임베딩 시각화
results = fob.compute_visualization(
    test_dataset,
    embeddings=image_embeddings_by_sample_id,
    brain_key="clip_embeddings",
    # num_dims=3,
    plot_points=True,
    verbose=True,
)

session = fo.launch_app(test_dataset)

Generating visualization...


/Users/bhc/opt/anaconda3/envs/datadrift/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ValueError: Found array with 0 sample(s) (shape=(0, 0)) while a minimum of 1 is required.